## Import librairies

In [ ]:
import pandas as pd
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

## Import data

In [ ]:
df = pd.read_csv('pl_data.csv')

In [ ]:
df

In [ ]:
df['category']

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['category'])
y = le.transform(df['category'])
print(y.shape)
print(list(le.inverse_transform([0, 1, 2])))

In [ ]:
X = []
index = 0

for files in df.images: 
    X.append(cv2.cvtColor(cv2.imread(files), cv2.COLOR_BGR2RGB))
    index = index+1

X = np.asarray(X)
print(X.shape)

In [ ]:
plt.imshow(X[0])

## Split into train and test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
plt.imshow(X_train[15])

## Build the model

In [ ]:
model = keras.models.Sequential()

#model.add( keras.layers.Input((28,28,1))) #input layer

model.add( keras.layers.Conv2D(8, (3,3),  activation='relu', input_shape = (100,100,3)) )#8 2dconvolutiv planes, kernel size 3*3
model.add( keras.layers.MaxPooling2D((2,2)))#reduce the image size on 4
model.add( keras.layers.Dropout(0.2))#deactivate randomely some neuron outputs (regularization and avoid overfitting)

model.add( keras.layers.Conv2D(16, (3,3), activation='relu') )
model.add( keras.layers.MaxPooling2D((2,2)))
model.add( keras.layers.Dropout(0.2))

model.add( keras.layers.Conv2D(32, (3,3), activation='relu') )
model.add( keras.layers.MaxPooling2D((2,2)))
model.add( keras.layers.Dropout(0.3))

model.add( keras.layers.Flatten()) 
model.add( keras.layers.Dense(400, activation='relu'))
model.add( keras.layers.Dropout(0.1))
model.add( keras.layers.Dense(400, activation='relu'))
model.add( keras.layers.Dropout(0.2))

model.add( keras.layers.Dense(3, activation='softmax'))

In [ ]:
model.summary()

model.compile(optimizer='adam',#function for get down the gradient
              loss='sparse_categorical_crossentropy',#loss function for classification
              metrics=['accuracy']) #the metric 

## Train the model

In [ ]:
%%time

batch_size  = 20
epochs      =  5
history = model.fit(  X_train, y_train,
                      batch_size      = batch_size,
                      epochs          = epochs,
                      verbose         = 1)
                    #   validation_data = (X_test, y_test))

In [ ]:
fig, ax1 = plt.subplots()

# Plot loss on the primary y-axis
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='tab:blue')
ax1.plot(history.history['loss'], label='Loss', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Create a secondary y-axis for accuracy
ax2 = ax1.twinx()
ax2.set_ylabel('Accuracy', color='tab:orange')
ax2.plot(history.history['accuracy'], label='Accuracy', color='tab:orange')
ax2.tick_params(axis='y', labelcolor='tab:orange')

# Add a title and show the plot
plt.title('Training Loss and Accuracy')
fig.tight_layout()
plt.show()

## Evaluation

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)

print(f'Test loss     : {score[0]:4.4f}')
print(f'Test accuracy : {score[1]:4.4f}')

In [ ]:
y_sigmoid = model.predict(X_test)
y_pred    = np.argmax(y_sigmoid, axis=-1)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
# import scikitplot as skplt 

print(classification_report(y_test,y_pred,digits = 4))
print(accuracy_score(y_test,y_pred))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(3), yticklabels=range(3))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

## Save and load the model and try with new data

In [ ]:
# YOU NEED TO CHANGE THE FILEPATH
model_path = "C:\\\\Users\\mdpdomco\\OneDrive - Haute Ecole de Namur-Liege-Luxembourg\\Cours\\M1\\systemes intelligents\\05_FruitsClassification\\reco.keras"
tf.keras.models.save_model(
    model,
    filepath = model_path,
    overwrite=True,
)

In [ ]:

model2 = keras.models.load_model(model_path)

## Save the model bis

In [ ]:
model_json = model.to_json()
with open("fruits_classifier.json", 'w') as json_file:
    json_file.write(model_json)
model.save("fruits_classifier.h5")

In [ ]:
from keras.models import model_from_json
json_file = open("fruits_classifier.json", "r")
model_json = json_file.read()
json_file.close()
model2 = model_from_json(model_json)
model2.load_weights("fruits_classifier.h5")

In [ ]:
NewData1 = cv2.cvtColor(cv2.imread('NouvelleBanane.jpg'), cv2.COLOR_BGR2RGB)
plt.imshow(NewData1)

In [ ]:
NewData1 = cv2.resize(NewData1,(100,100))
plt.imshow(NewData1)

In [ ]:
NewData2 = cv2.cvtColor(cv2.imread('NouvelleOrange.jpg'), cv2.COLOR_BGR2RGB)
plt.imshow(NewData2)

In [ ]:
NewData2 = cv2.resize(NewData2,(100,100))
plt.imshow(NewData2)

In [ ]:
NewData3 = cv2.cvtColor(cv2.imread('NouvellePomme.jpg'), cv2.COLOR_BGR2RGB)
plt.imshow(NewData3)

In [ ]:
NewData3 = cv2.resize(NewData3,(100,100))
plt.imshow(NewData3)

In [ ]:
NewData = np.stack((NewData1,NewData2,NewData3), axis = 0)

In [ ]:
NewData.shape

In [ ]:
y_sigmoid = model2.predict(NewData)
y_pred    = np.argmax(y_sigmoid, axis=-1)
print(y_pred)

# Next steps
adapt the code to classify the entire dataset available at https://github.com/Horea94/Fruit-Images-Dataset

Use the proposed train and test separation in the folders. 

first tackle only with the first word of each folder as class : "Apple Braeburn" -> "Apple" 
Then try to differentiate between the different categories of the same fruit.